### This is just to cross check the accuracy of the model with proper 5 fold cross validation method

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.model_selection import KFold
import random 
import torch 
from torch import nn 
from torch import optim 
from torch.utils.data import DataLoader,TensorDataset, random_split,SubsetRandomSampler,ConcatDataset
from torch.nn import functional as F
from torchvision import transforms
from torchsummary import summary
from tqdm.notebook import tqdm 
%matplotlib inline

Load the whole dataset into a vatiable, `Dataset`

Define the Neural Neural Architecture

Write loss function, optimizer,  epochs and all

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)

criterion = nn.CrossEntropyLoss()

EPOCHS = 40
batch_size = 16
k=5
splits =KFold(n_splits=5,shuffle=True,random_state=42)

Details = {}

### Training function 

In [ ]:
def training(model,trainloader,optimizer,lossFn,device):
    trainLoss = 0.0
    trainCorrect = 0
    model.train()
    for images, labels in tqdm(trainloader):
        images,labels = images.to(device),labels.to(device)
        
        optimizer.zero_grad()
        output = model(images) # input format [B.C,W,H]
        loss = lossFn(output,labels)
        loss.backward()
        optimizer.step()

        trainLoss += loss.item() 
        preds = output.argmax(dim=1)
        trainCorrect += (preds==labels).cpu().detach().sum().item()        
        # trainCorrect += (preds.cpu().detach()==labels.cpu().detach()).sum().item()
    return trainLoss,trainCorrect        

### Validation function